In [152]:
import pandas as pd 
import pymongo

## Connecting to DB

In [153]:
client = pymongo.MongoClient("localhost", 27017)
db = client["legume-choice"]
print(db.name)
# Extracting the projects data
projectsData =  db["projects"]
# Finding the number of projects
print(projectsData.count_documents({}))

legume-choice
3


## Extracting Data

In [154]:
# Extacting all projects
allProjects = []
for project in projectsData.find():
        allProjects.append(project)

# Example querying from the dataset

allProjects[0]["rawdata"]["agroEcoData"]


{'biofilters': [{'name': 'Rainfall (mm/year)',
   'label': 'rainfall',
   'minValue': 280,
   'maxValue': 4290,
   'value': 2175},
  {'name': 'Temperature (mean °C/month)',
   'label': 'temp',
   'minValue': 0,
   'maxValue': 45,
   'value': 14.5},
  {'name': 'Altitude (average masl)',
   'label': 'alt',
   'minValue': 0,
   'maxValue': 3800,
   'value': 1845},
  {'name': 'Soil pH (average)',
   'label': 'soilpH',
   'minValue': 4,
   'maxValue': 9,
   'value': 8.5}]}

## Extracting Agro Ecological Scores for all Projects

In [155]:
# Tabular AgroEco scores

def ExtractProjectAgroEcoData(project):
    projectID =  project["projectID"]
    projectName =  project["rawdata"]["projectInfo"]["projectName"]

    agroEcoData = project["rawdata"]["agroEcoData"]["biofilters"]

    row={"projectID":projectID,"projectName":projectName}


    for index in range(0,len(agroEcoData)):
        row[agroEcoData[index]["label"]]=agroEcoData[index]["value"]
        
    return row
def ExtractAllAgroEcoData(projects):
    agroEcoData=[]
    for project in projects:
        agroEcoData.append(ExtractProjectAgroEcoData(project=project))
    return pd.DataFrame(agroEcoData)


ExtractAllAgroEcoData(allProjects)


,projectID,projectName,rainfall,temp,alt,soilpH
0,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,wqeaafd,2175,14.5,1845,8.5
1,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Project 2,2186,28.5,1875,5.5
2,9c824f39-779c-446b-8d36-3f114fcdfeb9,Project 3,2276,27.5,1876,7.5


## Extracting all Data for Context Scores

In [156]:
def subsetScoresByVariables(scores, attributeLabel, participantLabel, typologyLabel, average):

    if (average==False):
        for score in scores:
            if (score["scoreType"]=="individual" and 
            score["attribute"]["label"]==attributeLabel and 
            score["participant"]["label"]==participantLabel and 
            score["typology"]["label"]==typologyLabel):
                 return score["score"]

    if (average==True):
        for score in scores:
            if (score["scoreType"]=="average" and 
            score["attribute"]["label"]==attributeLabel): 
                return score["score"]

    raise ValueError("Did not identify correct subset")
    

def ScoresForAllAttributes(scores, attributes, participant, typology, average):
    listOfScores=[]
    for attribute in attributes:
        listOfScores.append(subsetScoresByVariables(scores=scores, 
        attributeLabel=attribute["label"], 
        participantLabel=participant["label"], 
        typologyLabel=typology["label"], 
        average=average))

    return listOfScores


def projectContextScores(project):

    # Initialising Variables
    projectID =  project["projectID"]
    projectName =  project["rawdata"]["projectInfo"]["projectName"]


    allScores = project["rawdata"]["contextScores"]["scores"]
    attributes = project["rawdata"]["contextScores"]["attributes"]
    participants = project["rawdata"]["contextScores"]["participants"]
    typologies = project["rawdata"]["contextScores"]["typologies"]

    # Creating column names for dataframe
    typologyColumns = [ "t_"+typology["label"] for typology in typologies]
    participantColumns = [participant["label"] for participant in participants]
    allColumns = [typ+"_"+particip for typ in typologyColumns for particip in participantColumns]

    # Initialising empty array of scores
    scoresToReturn={}
    column=0
    scoresToReturn["projectName"] = [projectName for attribute in attributes]
    scoresToReturn["projectID"] = [projectID for attribute in attributes]
    for typology in typologies:
        for participant in participants:
            scoresToReturn[allColumns[column]]=ScoresForAllAttributes(scores=allScores, 
                                                                        attributes=attributes,
                                                                        participant=participant, 
                                                                        typology=typology, 
                                                                        average=False )
            column+=1
    
    scoresToReturn["average"]=ScoresForAllAttributes(scores=allScores, 
                                                        attributes=attributes,
                                                        participant=participants[1], 
                                                        typology=participants[1], 
                                                        average=True )
    return scoresToReturn

def ContextScoresAllProjectsList(projects):
    projectsCombined=[]
    #projectIndex=0
    for project in projects:
        projectsCombined.append(projectContextScores(project=project))
        #projectIndex+=1
    return(projectsCombined)

def ContextScoresAllProjects(projects):
    contextScoresList = ContextScoresAllProjectsList(allProjects)
    if len(contextScoresList)==1:
        return pd.DataFrame(contextScoresList)
    if len(contextScoresList)>1:
        contextScoresDF=pd.DataFrame(contextScoresList[0])
        for scoreItem in contextScoresList[1:]:
            contextScoresDF = contextScoresDF.append(pd.DataFrame(scoreItem))
    return contextScoresDF
ContextScoresAllProjects(projects=allProjects)


,projectName,projectID,t_low_farmer,t_low_expert,t_med_farmer,t_med_expert,t_high_farmer,t_high_expert,average
0,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,1,2,3,1,4,2,2.166667
1,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,2,3,1,2,4,4,2.666667
2,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,1,2,4,1,2,4,2.333333
3,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,2,3,4,4,4,4,3.500000
4,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,4,1,4,0,3,4,2.666667
5,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,1,1,0,0,1,0,0.500000
6,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,3,0,4,0,0,0,1.166667
0,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,3,2,4,1,4,2,2.666667
1,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,1,1,1,2,4,4,2.166667
2,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,4,0,4,1,0,4,2.166667


## Extracting Data for PairWise Rankings

### Individual Selections

In [157]:
def pairWiseSelectionsProjectGender(project,gender):

    selections = project["rawdata"]["pairWiseScores"][gender]["pairwiseSelection"]
    projectName= project["rawdata"]["projectInfo"]["projectName"]
    projectID= project["projectID"]

    selectionsToReturn=[]

    for selection in selections:
        selectionToAppend={
            "projectName": projectName,
            "projectID":projectID,
            "option1": selection["funct1"]["name"],
            "option2": selection["funct2"]["name"],
            gender+"Choice": selection["value"]["name"],
        }
        selectionsToReturn.append(selectionToAppend)
    return pd.DataFrame(selectionsToReturn)

def pairWiseSelectionsProject(project):
    male=pairWiseSelectionsProjectGender(project=project,gender="male")
    female=pairWiseSelectionsProjectGender(project=project,gender="female")

    allSelections = male
    allSelections["femaleChoice"] = female["femaleChoice"]
    return allSelections

def pairWiseSelectionsAllProjects(projects):
    if len(projects)==1:
        return pairWiseSelectionsProject(projects)
    if len(projects)>1:
        pairWiseScoresDF=pairWiseSelectionsProject(projects[0])
        for project in projects[1:]:
            pairWiseScoresDF = pairWiseScoresDF.append(pairWiseSelectionsProject(project))
    return pairWiseScoresDF
pairWiseSelectionsAllProjects(projects=allProjects)



,projectName,projectID,option1,option2,maleChoice,femaleChoice
0,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,Feed,Feed,Feed
1,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,Income,Food,Food
2,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,Erosion Control,Food,Erosion Control
3,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,Fuel,Fuel,Food
4,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,Soil Fertility,Food,Soil Fertility
5,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,Income,Income,Feed
6,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,Erosion Control,Feed,Erosion Control
7,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,Fuel,Fuel,Fuel
8,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,Soil Fertility,Soil Fertility,Soil Fertility
9,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Income,Erosion Control,Income,Erosion Control


### Averages

In [158]:
def pairWiseSummaryScores(project):
    projectID = project["projectID"]
    projectName = project["rawdata"]["projectInfo"]["projectName"]
    maleTotals = project["rawdata"]["pairWiseScores"]["male"]["totals"]
    femaleTotals = project["rawdata"]["pairWiseScores"]["female"]["totals"]
    averages = project["rawdata"]["pairWiseScores"]["averages"]


    pairWiseAverages = []
    for scoreIndex in range(0,len(maleTotals)):
        scoresToAppend = {
            "projectName":projectName,
            "projectID":projectID,
            "attribute": maleTotals[scoreIndex]["attribute"]["name"],
            "countMale": maleTotals[scoreIndex]["value"],
            "countFemale": femaleTotals[scoreIndex]["value"],
            "average": averages[scoreIndex]["value"],
        }
        pairWiseAverages.append(scoresToAppend)

    return pd.DataFrame(pairWiseAverages)

def pairWiseSelectionSummaryAllProjects(projects):
    if len(projects)==1:
        return pairWiseSummaryScores(projects)
    if len(projects)>1:
        pairWiseSummaryScoresDF=pairWiseSummaryScores(projects[0])
        for project in projects[1:]:
            pairWiseSummaryScoresDF = pairWiseSummaryScoresDF.append(pairWiseSummaryScores(project))
    return pairWiseSummaryScoresDF
pairWiseSelectionSummaryAllProjects(projects=allProjects)


,projectName,projectID,attribute,countMale,countFemale,average
0,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,3,2,2.5
1,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,2,2,2.0
2,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Income,3,1,2.0
3,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Erosion Control,0,3,1.5
4,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Fuel,4,4,4.0
5,wqeaafd,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Soil Fertility,3,3,3.0
0,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Food,0,3,1.5
1,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Feed,3,2,2.5
2,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Income,3,1,2.0
3,Project 2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Erosion Control,1,1,1.0


## Participatory Matrix Scores

In [159]:

# Extracting farmer data
def participatoryMatrixScoresProject(project):
    #project=allProjects[0]
    projectID = project["projectID"]
    projectName = project["rawdata"]["projectInfo"]["projectName"]
    farmers = project["rawdata"]["participatoryMatrixScores"]["farmers"]
    farmersClean=[]
    for farmer in farmers:
        newFarmer={}
        newFarmer["projectID"]=projectID
        newFarmer["projectName"]=projectName
        newFarmer["gender"]=farmer["gender"]
        newFarmer["typology"]=farmer["typology"]
        
        for selection in farmer["selections"]:
            newFarmer[selection["label"]]=selection["score"]
        newFarmer["total"]=farmer["total"]
        farmersClean.append(newFarmer)
    return pd.DataFrame(farmersClean)


def participatoryMatrixAllProjects(projects):
    if len(projects)==1:
        return participatoryMatrixScoresProject(projects)
    if len(projects)>1:
        participatoryMatrixScoresDF=participatoryMatrixScoresProject(projects[0])
        for project in projects[1:]:
            participatoryMatrixScoresDF = participatoryMatrixScoresDF.append(participatoryMatrixScoresProject(project))
    return participatoryMatrixScoresDF
participatoryMatrixAllProjects(projects=allProjects)

,projectID,projectName,gender,typology,food,feed,income,erosion-control,fuel,soil-fertility,total
0,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,wqeaafd,Male,Low,2,3,5,4,2,4,20
1,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,wqeaafd,Female,Medium,2,4,5,2,4,3,20
2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,wqeaafd,Male,High,8,0,4,2,5,1,20
3,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,wqeaafd,Female,High,3,7,1,5,1,3,20
0,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Project 2,Female,Low,0,5,5,4,2,4,20
1,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Project 2,Male,High,0,4,7,2,4,3,20
2,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Project 2,Female,Medium,6,2,4,2,3,3,20
3,d4e74b50-6b07-4bb2-9be7-af7f52e4947d,Project 2,Male,Low,1,9,0,6,1,3,20
0,9c824f39-779c-446b-8d36-3f114fcdfeb9,Project 3,Female,Medium,2,4,3,3,5,3,20
1,9c824f39-779c-446b-8d36-3f114fcdfeb9,Project 3,Male,High,4,2,3,5,3,3,20
